In [1]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
import json
import re
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from tqdm import tqdm

STOP_WORDS = set(stopwords.words("russian"))

def clean_text(text: str, lower: bool = True, remove_stopwords: bool = True) -> str:
    """
    Базовая очистка текста:
    - удаляет HTML-разметку, служебные символы
    - стандартизирует пробелы
    - приводит к нижнему регистру
    - фильтрует стоп-слова
    """
    if not isinstance(text, str):
        return ""

    # Удаляем HTML
    text = BeautifulSoup(text, "html.parser").get_text()

    # Убираем неразрывные пробелы и управляющие символы
    text = text.replace("\xa0", " ")
    text = re.sub(r"[\r\t]+", " ", text)

    # Приведение к нижнему регистру
    if lower:
        text = text.lower()

    # Стандартизация пробелов
    text = re.sub(r"\s+", " ", text).strip()

    # Фильтрация стоп-слов (по желанию)
    if remove_stopwords:
        words = [w for w in text.split() if w not in STOP_WORDS]
        text = " ".join(words)

    return text


def process_jsonl(input_file: str, output_file: str):
    """
    Читает JSONL, очищает текст и перезаписывает title/text.
    """
    total = 0
    with open(input_file, "r", encoding="utf-8") as fin, \
         open(output_file, "w", encoding="utf-8") as fout:
        for line in tqdm(fin, desc="Очистка текста"):
            record = json.loads(line)

            # Очистка
            record["title"] = clean_text(record.get("title", ""))
            record["text"] = clean_text(record.get("text", ""))

            # Запись очищенной записи
            json.dump(record, fout, ensure_ascii=False)
            fout.write("\n")

            total += 1

    print(f"Очистка завершена. Сохранено {total} записей в {output_file}")


In [10]:
input_file = "1_news_corpus_1k.jsonl"
output_file = "2_news_corpus_clean.jsonl"
process_jsonl(input_file, output_file)


Очистка текста: 1000it [00:00, 2576.87it/s]

✅ Очистка завершена. Сохранено 1000 записей в 2_news_corpus_clean.jsonl
